In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import plotly.express as px

In [36]:
df_CONTEO = pd.read_csv('C:/Users/maria/Downloads/database.csv', 
                 sep = ',', parse_dates= True,
                        thousands=',', decimal='.',
                      
                 )
df_CONTEO.head()

,article_id,full_name,sex,birth_year,city,state,country,continent,latitude,longitude,occupation,industry,domain,article_languages,page_views,average_views,historical_popularity_index
0,308,Aristotle,Male,-384,Stageira,NaN,Greece,Europe,40.33333,23.50000,Philosopher,Philosophy,Humanities,152,56355172,370758,31.9938
1,22954,Plato,Male,-427,Athens,NaN,Greece,Europe,37.96667,23.71667,Philosopher,Philosophy,Humanities,142,46812003,329662,31.9888
2,1095706,Jesus Christ,Male,-4,Judea,NaN,Israel,Asia,32.50000,34.90000,Religious Figure,Religion,Institutions,214,60299092,281771,31.8981
3,25664190,Socrates,Male,-469,Athens,NaN,Greece,Europe,37.96667,23.71667,Philosopher,Philosophy,Humanities,137,40307143,294213,31.6521
4,783,Alexander the Great,Male,-356,Pella,NaN,Greece,Europe,40.80000,22.51667,Military Personnel,Military,Institutions,138,48358148,350421,31.5840


In [37]:
for col in df_CONTEO.columns:
    num_missing = df_CONTEO[col].isnull().sum()
    pct_missing = np.mean(df_CONTEO[col].isnull())
    #print(col," - ",round(pct_missing*100, 2),"% - ",num_missing)
    print('{} - {}% - {}'.format(col,round(pct_missing*100, 2),num_missing))

article_id - 0.0% - 0
full_name - 0.0% - 0
sex - 0.0% - 0
birth_year - 0.0% - 0
city - 0.0% - 0
state - 80.85% - 9169
country - 0.29% - 33
continent - 0.26% - 30
latitude - 9.23% - 1047
longitude - 9.23% - 1047
occupation - 0.0% - 0
industry - 0.0% - 0
domain - 0.0% - 0
article_languages - 0.0% - 0
page_views - 0.0% - 0
average_views - 0.0% - 0
historical_popularity_index - 0.0% - 0


Primero se visualizara los valores faltantes para los porcentajes que no son 0.0%

Para el tipo de visualización y modismo a utilizar no se necesitara hacer limpieza de datos, ya que las variables a utilizar no tienen valores perdidos. A pesar de eso se hará una pequeña limpieza para una de las posibles variables a utilizar.

In [38]:
df1 = df_CONTEO.copy()

In [39]:
df1[df1['state'].isnull()]

,article_id,full_name,sex,birth_year,city,state,country,continent,latitude,longitude,occupation,industry,domain,article_languages,page_views,average_views,historical_popularity_index
0,308,Aristotle,Male,-384,Stageira,NaN,Greece,Europe,40.33333,23.50000,Philosopher,Philosophy,Humanities,152,56355172,370758,31.9938
1,22954,Plato,Male,-427,Athens,NaN,Greece,Europe,37.96667,23.71667,Philosopher,Philosophy,Humanities,142,46812003,329662,31.9888
2,1095706,Jesus Christ,Male,-4,Judea,NaN,Israel,Asia,32.50000,34.90000,Religious Figure,Religion,Institutions,214,60299092,281771,31.8981
3,25664190,Socrates,Male,-469,Athens,NaN,Greece,Europe,37.96667,23.71667,Philosopher,Philosophy,Humanities,137,40307143,294213,31.6521
4,783,Alexander the Great,Male,-356,Pella,NaN,Greece,Europe,40.80000,22.51667,Military Personnel,Military,Institutions,138,48358148,350421,31.5840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11335,32941975,Jetro Willems,Male,1994,Rotterdam,NaN,Netherlands,Europe,51.91667,4.50000,Soccer Player,Team Sports,Sports,27,577913,21404,11.3956
11336,5891391,Sean St Ledger,Male,1984,Solihull,NaN,United Kingdom,Europe,52.41290,-1.77209,Soccer Player,Team Sports,Sports,28,380965,13606,11.1346
11337,5322451,Saina Nehwal,Female,1990,Hisar,NaN,India,Asia,29.16670,75.71670,Athlete,Individual Sports,Sports,26,2255010,86731,10.6122
11338,33525706,Rūta Meilutytė,Female,1997,Kaunas,NaN,Lithuania,Europe,54.90000,23.88611,Swimmer,Individual Sports,Sports,32,342719,10710,10.3821


In [41]:
df1['state'] = df_CONTEO['state'].fillna('Sin nombre')

Los datos de Pantheon 1.0 mien la popularidad global de los personajes históricos utilizando dos medidas. La primera es el número de diferentes ediciones de idioma de Wikipedia que tienen un artículo sobre un personaje histórico. La segunda contiene el índice de popularidad histórica añadiendo información sobre la edad del carácter histórico, la concentración de vistas de página entre diferentes idiomas, el coeficiente de variación en las vistas de página y el número de vistas de página en idiomas distintos del inglés.

El modismo tiene como objetivo analizar y ver las relaciones que tienen las diferentes variables entre ellas. Intentando responder a la pregunta ¿Qué ocupaciones o industrias son las más populares? en la la popularidad global de los personajes históricos. Las variables a utilizar seran sexo, industria, ocupación y vistos de la pagina.

#### QUE 

Basados en el Framework de Tamara, evidenciamos que el Dataset cuenta con atributos categóricos para  las variables excepto popularidad historica y el sexo. visto de la pagina son variables ordenadas cuantitativas, el tipo de Data Set utilizado será un Treemap que contienen clasificaciones, son datos estáticos, es decir, que no cambian en el tiempo. Tiene 1 atributo cuantitativo en los nodos de las hojas.

#### PORQUE

En el caso de los verbos utilizados para la descripción de la visualización, se utilizaron el consumir y presentar,por medio del Treemaps el análisis básico de las proporciones de la popularidad global de los personajes históricos, se buscara la segmentación de esta popularidad a traves de la industria y las ocupaciones. Todo esto con el fin de descubrir,analizar y comparar estas variables.

En el caso de los sustantivos que se utilizaron para las variables se tienen en cuenta la proporción y separabilidad. 
Adicionalmente, la visualización tiene estructura jerárquica donde les asigna un tamaño y un orden en función de la variable cuantitativa. Su searh puede estar referenciado con un locate, dado que su target se conoce por lo cual se puede identificar que industrias son las mas populares a traves de su popularidad historica, que ocupaciones son las mas reconocidas y además, si hay más personas de genero femenino o masculino.


#### COMO

Fueron estructurados en forma de árbol, por jerarquía, como un conjunto de rectángulos anidados. A cada grupo se le asigna un rectángulo, que después se combina con rectángulos más pequeños que representan subgrupos. El tamaño y el color se usan para mostrar dimensiones numéricas independientes de los datos. Siendo el primer conjunto la industria, luego se combina con las ocupaciones y por ultimo el sexo. 

In [23]:
fig = px.treemap(df1, path=['industry', 'occupation','sex'], 
                 values='historical_popularity_index', 
                 color='occupation')
  
fig.show()

#### MARCAS Y CANALES 

•	Los canales utilizados en las visualizaciones serán el área, el color, y el tamaño para cada atributo, el color representa la categoría será topológico. 

•	Las marcas serán las figuras (rectangulos) y las áreas.


#### MEJORAS

En general la visualización me parece muy buena ya que se ve muy claramente la jerarquía pues cada conjunto de rectángulos del mismo nivel de jerarquía representa una columna y cada rectángulo individual o más pequeño representa una categoría de una columna. Además, el canal de color es muy bueno para la visualización. Por lo cual es facil identificar y analizar el modismo propuesto

Como conclusión para el modismo podemos notar que la industria con más popularidad global para los personajes históricos es la gobierno, teniendo una ocupación sobresaliente de políticos y como muy obvio que para esta industria el 80% de los personajes sean hombres. La segunda ocupación más destacada es la de cine y teatro con la ocupación de actor como segmentación, pero para este caso, aunque no es mucha la diferencia se ve que las mujeres tienen una proporción en esta industria.  Por lo cual se puede llegar a la conclusión de que las mujeres no han sido muy reconocidas a través del tiempo ya que en la sociedad tradicional la mujer se quedaba en el ámbito doméstico y a las labores de crianza. 

Sin embargo, hubo excepciones, hubo mujeres que lograron superar esas barreras, desarrollar una profesión y dedicarse a su vocación, donde han luchado por conseguir un espacio destacado en un mundo que parecía ser dominado por los hombres. 

Adicionalmente, sera hará una visualización que muestre lo mismo solo que utilizando la variable estado

In [26]:
fig = px.treemap(df1, path=['industry','state','occupation','sex'], 
                 values='historical_popularity_index', 
                 color='occupation')
  
fig.show()

Como se puede ver el estado no reconocido es de los que más información tiene para la popularidad global de los personajes históricos.  Aunque a esta visualización si le cambiaria un poco el color ya que en algunos estados, industrias y ocupaciones utiliza el mismo para todos. 